## Tutorial: Increasing the Predictive Power of Your Machine Learning Models with Stacking Ensembles

Whoever accompanies competitions knows that one of the most important things is to know how to put together several models to create a powerful solution. Several people have already asked me, by e-mail or in the presentations I made, about ensembles. This is an important issue not only for competitions, but also for real cases where you want to extract as much performance as possible from the models.

Ensembles are sets of models that offer better performance than each model that composes it.

So in this article I want to exemplify the best way I know of creating ensembles: stacking. This is a method I have used in all competitions that have had good results.

Before you begin, a tip: It's important to think of Machine Learning as "processes." Here we will not only test models in a database, but we will test processes, pipelines of methods applied to the data to know what the results are.

This material is part of the presentation I will make (or did) on 30/11 at the PyData Meetup São Paulo. The original Jupyter Notebook is available at this link.


In [1]:
import pandas as pd
import numpy as np

## Loading data

The data used are commercial real estate transactions in the city of Ames, Iowa. Our goal is to predict the selling price of a house by feeding the model with the features. This data is also the subject of a competition at Kaggle. The training and test data can be found at: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/

Something very important is to explore the data for ideas of features and methods to validate the models. As the focus of this material is stacking, I'll skip this part. Nevertheless, I think it is important to clarify that we have two basic types of variables in this data: categorical and numerical.

Categorical variables have levels that can not be ordered. In some cases it is possible to think of ways to order them, as a variable that describes whether a street is paved or not. In this case one can think that the paved street will value the property.

Numeric variables can receive any continuous value. An example would be the size of the terrain.

In this cell we load the data and create a DataFrame with the features (X) and a Series with the sale price (SalePrice, y) that is our target.

In [2]:
train = pd.read_csv('train.csv', index_col='Id')
X, y = train.drop('SalePrice', axis=1), train.SalePrice.copy()
train.head().iloc[:, :5]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street
Id,,,,,
1,60,RL,65.0,8450,Pave
2,20,RL,80.0,9600,Pave
3,60,RL,68.0,11250,Pave
4,70,RL,60.0,9550,Pave
5,60,RL,84.0,14260,Pave


## Sets of Variables (Features)

One way to get different models is to vary the data representation used to train them. That is why our first step is to build these representations.

### Auxiliary Functions

The metric suggested by Kaggle to evaluate the models is RMSLE, this error takes into account the difference between the logarithm of the predictions and the target. It is possible to think of this error as an approximation of the percentage error of the model, but with more interesting properties from the mathematical point of view.

I created three functions to calculate the error. We are going to do transformations of the variable y, so to make our work easier, I decided to create custom functions for each transformation, thus avoiding some confusion when transforming them to compute the error in a single function.

To use some Scikit-learn functions that will help keep our code cleaner and more efficient, we need to create the error functions in the way that the module requires. In this case, the pecse function receives a trained model, the features and the target. It computes the predictions and should return a number, which is the value of the error metric.

The last cell line creates a cross-validation object from scikit-learn. He will take care of dividing the data for us. This subject deserves a number of articles, but basically it is a way to divide the data into N parts, and repeatedly use N-1 parts as training data, and the remaining part as test data. For example, if we have the parts [1, 2, 3, 4, 5], in one of the iterations we can train using parts [1, 2, 3, 4] and validate in part 5.

Although this is a time series, Kaggle has decided to treat as independent random data, so we will do this simple cross validation.

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor

def rmsle(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

def rmsle_log_y(estimator, X, y):
    p = estimator.predict(X)
    return np.sqrt(mean_squared_error(y, p))

def rmsle_sqrt_y(estimator, X, y):
    p = estimator.predict(X)
    y = np.power(y, 2)
    p = np.power(p, 2)
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

kf = KFold(n_splits=5, shuffle=True, random_state=1)

### Feature set 1: "numeric" variables

The first set of features we will have will be a simple selection of the originally numeric data variables. To do this, just select all the columns that have variables with integers or floating point. Also, to replace the null values ​​(scikit-learn requires replacement), I decided to put the -1 value. Since we are going to use only tree-based models with this unique data, there is no need to worry too much about it for our purposes.

After storing this data in variable X1, we see that there are 36 columns. Already in this part I want to train a model of Random Forest inside the cross-validation, to know how it would leave alone in the original data. For this I used the cross_val_score function. It is a feature that makes it much easier to cross-validate with scikit-learn. Just put a template and the data. In this case I chose to specify a validation scheme and a custom error metric.

This function returns a list with the errors of each iteration of the cross validation, so I averaged to know the average error of the parts.

In [4]:
X1 = X.select_dtypes(include=[np.number]).fillna(-1)
print('Dims', X1.shape)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, y, cv=kf, scoring=rmsle).mean()
print('RMSLE:', error)

Dims (1460, 36)
RMSLE: 0.14582352618


### Feature set 2: Ordinal Encoding Categorical

Now let's create another set of features, this time adding the categorical variables. There are several ways to encode this type of variable for the models, one of which is using an ordinal format. This simply means replacing each original value with sequential numbers. On some models this can be problematic as they will try to capture some order relation in values they may not have. In our case, with models based on decision trees, this problem is almost non-existent.

After coding in this way, we ran the cross-validation again, now on these new data.

In [5]:
from sklearn.preprocessing import LabelEncoder

X2 = X.copy()
for col in X2.columns:
    if X2[col].dtype == object:
        enc = LabelEncoder()
        X2[col] = enc.fit_transform(X[col].fillna('Missing'))

print('Dims', X2.shape)
X2.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, y, cv=kf, scoring=rmsle).mean()
print('RMSLE:', error)

Dims (1460, 79)
RMSLE: 0.143837364859


### Bonus: OneHot Encoding

The most popular way to code categorical variables is One Hot Encoding. Basically consists of transforming each value of the variable into a column whose new value will be 1 if the variable has that value in a given example, or 0 if not. There are indications that decision trees do not process this kind of representation so well, but in some practical cases I've seen this work better than the ordinal, so look at this as another tool.

This method creates more than 200 new columns, which makes the training process slower, so I decided to leave the cross-validation line commented out. If you want to see the result, just run it without the old woman's game.

In [6]:
#from sklearn.preprocessing import OneHotEncoder
X3 = X.copy()
cats = []
for col in X3.columns:
    if X3[col].dtype == object:
        X3 = X3.join(pd.get_dummies(X3[col], prefix=col), how='left')
        X3.drop(col, axis=1, inplace=True)
    

print('Dims', X3.shape)
X3.fillna(-1, inplace=True)
model = RandomForestRegressor(n_estimators=1000, random_state=0)
#cross_val_score(model, X3, y, cv=kf, scoring=rmsle).mean()

Dims (1460, 288)


## Target Transformations

An interesting way to create diversity, and sometimes even better performance, in a regression case is to transform the variable we are trying to predict. In this case we will test two transformations: logarithm and square root.

### Log

It is possible to see that trying to predict the logarithm of the price gives us a better result. This happens not only because the model captures different patterns, but also because we use a metric based on the difference of logarithms.

In [7]:
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, X1, log-target RMSLE:', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('RF, X2, log-target RMSLE:', error)

RF, X1, log-target RMSLE: 0.14518580749
RF, X2, log-target RMSLE: 0.14207134495


### Square root

This transformation also gives us a better result than using the variable in its original state. One of the suggestions of the reason we see this effect is that these transformations cause the variable y to have a distribution closer to normal, which facilitates the work of the model.

In [8]:
model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, X1, sqrt-target RMSLE:', error)

model = RandomForestRegressor(n_estimators=1000, random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('RF, X2, sqrt-target RMSLE:', error)

RF, X1, sqrt-target RMSLE: 0.145652934484
RF, X2, sqrt-target RMSLE: 0.143004600132


## Generating models with different models / algorithms

Another way to generate diversity for the ensemble is to generate different models. In this case I will use my preferred model, the GBM. This is also based on decision trees, but basically trains each tree sequentially focusing on the mistakes made by the previous ones.

In the cells below you can see the performance of this model in the feature sets and transformations we use with Random Forest. We see that it brings a significant improvement, capturing better the patterns of the relationship between the variables and the sale price of the real estate.

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
                
model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, X1, log-target RMSLE:', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.log1p(y), cv=kf, scoring=rmsle_log_y).mean()
print('GBM, X2, log-target RMSLE:', error)

GBM, X1, log-target RMSLE: 0.133492454914
GBM, X2, log-target RMSLE: 0.129806890482


In [10]:
from sklearn.ensemble import GradientBoostingRegressor
                
model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X1, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X1, sqrt-target RMSLE:', error)

model = GradientBoostingRegressor(random_state=0)
error = cross_val_score(model, X2, np.sqrt(y), cv=kf, scoring=rmsle_sqrt_y).mean()
print('GBM, X2, sqrt-target RMSLE:', error)

GBM, X1, sqrt-target RMSLE: 0.134258972813
GBM, X2, sqrt-target RMSLE: 0.130919235682


### Adjusting hyperparameters

To simplify the example and focus on the ensemble part, I will not adjust the hyperparameters of the model. Hyperparameters are the attributes of the model (such as the depth of the decision trees) that need to be adjusted using separate validation data or a cross-validation cycle.

It is good to know that not always the best models form the best ensembles. It is important to have powerful models when stacking, but we must also remember that it is important to have diversity. Sometimes some models that have a higher error can capture different patterns of the best models and therefore contribute to the ensemble.

If you decide to adjust the hyperparameters, it is important to place it within the cross-validation cycle that we will see in the next step.

## Stacking

All we did above is so we can create our ensemble. This is the time to put together the methods used to improve the predictive power of our models.

Stacking is a way of doing the ensemble in which we use models to make predictions, and then we use these predictions as features in new models, in what can be called the "second level." You can do this process several times, but at each level the return on performance with respect to the required computation is less.

In this phase we need two cycles of cross-validation: external and internal. Inside, we will train the models on the original data and make the predictions. On the outside, we will train the model using the first step predictions as features.

At each step of internal cross validation, we will save the predictions for the part of the data that is used as validation. This way we will have predictions for all examples of our training data. In addition, we will train a model in the original training data so that we can make predictions for the test data.

In the external cycle we will train a model in the predictions generated by the internal cycle, and the features of the validation data will be the predictions of the first level models in the test data.

In our specific case, we will create predictions using all combinations of models (RF and GBM), target (log and square root) transformations and feature sets (X1 and X2). We will not use X3 because it would take a lot of time to train, and for the purposes of demonstrating the method these two will be enough.

In the end we will have predictions of 8 models in the first level. In the second level I used a regularized linear regression (Ridge). Having these predictions we can compute the error in the data outside of our internal training and validation samples, which will give us a reliable estimate of the ensemble error.

In [13]:
from itertools import product
from sklearn.linear_model import Ridge

kf_out = KFold(n_splits=5, shuffle=True, random_state=1)
kf_in = KFold(n_splits=5, shuffle=True, random_state=2)

cv_mean = []
for fold, (tr, ts) in enumerate(kf_out.split(X, y)):
    X1_train, X1_test = X1.iloc[tr], X1.iloc[ts]
    X2_train, X2_test = X2.iloc[tr], X2.iloc[ts]
    y_train, y_test = y.iloc[tr], y.iloc[ts]
    
    modelos = [GradientBoostingRegressor(random_state=0), RandomForestRegressor(random_state=0)]
    targets = [np.log1p, np.sqrt]
    feature_sets = [(X1_train, X1_test), (X2_train, X2_test)]
    
    
    predictions_cv = []
    predictions_test = []
    for model, target, feature_set in product(modelos, targets, feature_sets):
        predictions_cv.append(cross_val_predict(model, feature_set[0], target(y_train), cv=kf_in).reshape(-1,1))
        model.fit(feature_set[0], target(y_train))
        ptest = model.predict(feature_set[1])
        predictions_test.append(ptest.reshape(-1,1))
    
    predictions_cv = np.concatenate(predictions_cv, axis=1)
    predictions_test = np.concatenate(predictions_test, axis=1)
    
    stacker = Ridge()
    stacker.fit(predictions_cv, np.log1p(y_train))
    error = rmsle_log_y(stacker, predictions_test, np.log1p(y_test))
    cv_mean.append(error)
    print('RMSLE Fold %d - RMSLE %.4f' % (fold, error))
    
print('RMSLE CV5 %.4f' % np.mean(cv_mean))
    


RMSLE Fold 0 - RMSLE 0.1248
RMSLE Fold 1 - RMSLE 0.1449
RMSLE Fold 2 - RMSLE 0.1257
RMSLE Fold 3 - RMSLE 0.1409
RMSLE Fold 4 - RMSLE 0.1087
RMSLE CV5 0.1290


As we can see, our best first level model is the GBM trained with the log transformation in the X2 data, which reaches the error of 0.1288. Our ensemble reaches the value of 0,1290. An improvement of 0.62%.

The purpose of this article was to demonstrate the method, without worrying too much about the end performance. An ensemble made for performance improvement may present a more significant result.

In some cases, such as in investment funds or healthcare applications, a small improvement can have a very significant result in the real world, which justifies the creation of a more complete solution using stacking.

As always in the Machine Learning application, nothing is a guarantee of success, but this method is one of the most consistent in offering an improvement.